Import modules

In [0]:
import tensorflow as tf
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

데이터 불러오기, 결측치 제거, 변수 선택

In [122]:
# 데이터 불러오기

# tistory 변수 list
# record_time	round	distance	humidity	weight_ctrl	dan	lane	bok	age	yeon	jk_weight	weight	location	rating	race_grade	
# weather1	weather2	weather3	weather4	weather5	nation1	nation2	nation3	nation4	nation5	nation6	nation7	nation8	sex	cure_1mnth	date	rank

data = pd.read_csv("tistory.csv",encoding='utf-8')
print("전체 데이터 개수 : ",len(data))


# distance별로 데이터 나누기
'''
gb_distance = data.groupby(['distance'])
gb_distance=[gb_distance.get_group(x) for x in gb_distance.groups]

# distance list
distance_list = list(set(data['distance']))
distance_list.sort()

for i in range(len(gb_distance)):
  print(distance_list[i] ,"m 거리에서 데이터 개수 : ",len(gb_distance[i]))


# 데이터 가장 많은 Dataframe 선택  

data=gb_distance[2]
'''

# tistory X,y
y = data.loc[:,'record_time']
X = data.loc[:, ['humidity','weight_ctrl','dan','lane','bok','age','yeon','jk_weight','weight','location','rating','race_grade','cure_1mnth','data','rank','distance']]

# 
X = X.as_matrix()
y = y.as_matrix()


전체 데이터 개수 :  116779


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [0]:
# 설정하는 부분

learning_rate = 0.01
training_epochs = 200
batch_size = 128


In [124]:
# Train , Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

train_size = len(X_train)
print("train_size : ",train_size)

size_x = len(X_train[0])
print("size_x : ",size_x)



train_size :  81745
size_x :  16


In [125]:
# Normalize 
X_mean = []
X_std = []

for i in range(X_train.shape[1]):
  X_mean.append(np.mean(X_train[:,i]))
  X_std.append(np.std(X_train[:,i]))
  print(i+1,"번째 변수의 X_mean:",'{:.3f}'.format(X_mean[i]),", X_std:",'{:.3f}'.format(X_std[i]))
  X_train[:,i] = (X_train[:,i]-X_mean[i])/X_std[i]
  X_test[:,i] = (X_test[:,i]-X_mean[i])/X_std[i]

  


1 번째 변수의 X_mean: 7.343 , X_std: 4.352
2 번째 변수의 X_mean: 0.027 , X_std: 0.162
3 번째 변수의 X_mean: 42.202 , X_std: 58.540
4 번째 변수의 X_mean: 6.255 , X_std: 3.481
5 번째 변수의 X_mean: 38.697 , X_std: 106.028
6 번째 변수의 X_mean: 3.644 , X_std: 1.064
7 번째 변수의 X_mean: 6.742 , X_std: 7.861
8 번째 변수의 X_mean: 53.712 , X_std: 1.807
9 번째 변수의 X_mean: 473.589 , X_std: 29.720
10 번째 변수의 X_mean: 0.577 , X_std: 0.494
11 번째 변수의 X_mean: 3.938 , X_std: 1.529
12 번째 변수의 X_mean: 3.892 , X_std: 1.580
13 번째 변수의 X_mean: 0.696 , X_std: 0.460
14 번째 변수의 X_mean: nan , X_std: nan
15 번째 변수의 X_mean: nan , X_std: nan
16 번째 변수의 X_mean: 1367.948 , X_std: 268.177


In [0]:
# Tensorflow

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, size_x])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)
phase = tf.placeholder(tf.bool, name='phase')

# First Layer

W1 = tf.get_variable(shape=[size_x, 128], name='weight1', initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([128]))
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)
layer1 = tf.contrib.layers.batch_norm(layer1, center=True, scale=True, 
                                          is_training=phase)

# Second Layer

W2 = tf.get_variable(shape=[128, 64], name='weight2', initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
layer2 = tf.nn.dropout(layer2, keep_prob=keep_prob)
layer2 = tf.contrib.layers.batch_norm(layer2, center=True, scale=True, 
                                          is_training=phase)
# Third Layer

W3 = tf.get_variable(shape=[64, 32], name='weight3', initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([32]))
layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)
layer3 = tf.nn.dropout(layer3, keep_prob=keep_prob)
layer3 = tf.contrib.layers.batch_norm(layer3, center=True, scale=True, 
                                          is_training=phase)
# Predict

W4 = tf.get_variable(shape=[32, 1], name='weight4', initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([1]))
hypothesis = tf.matmul(layer3, W4) + b4
hypothesis = tf.contrib.layers.batch_norm(hypothesis, center=True, scale=True, 
                                          is_training=phase)



In [0]:
# Cost 계산

cost = tf.reduce_mean(tf.square(hypothesis - Y))
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)



In [130]:
# 계산 시작

sess = tf.Session()
sess.run(tf.global_variables_initializer())
avg_cost_prev=0
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_size / batch_size)

    for i in range(total_batch):
        # batch_size로 X,y 개수 나눠주기
        batch_xs = X_train[i*batch_size:(i+1)*batch_size]
        batch_ys = y_train[i*batch_size:(i+1)*batch_size].reshape(batch_size,1)
        
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:0.7,phase:True}
        c, _,hyp,realy = sess.run([cost, train,hypothesis,Y], feed_dict=feed_dict)
        avg_cost += c/total_batch
        
    
    # 이전 epoch와 차이가 0.001 이하면
    if abs(avg_cost - avg_cost_prev) <= 0.001:
      break

    # 20 epoch마다 cost
    avg_cost_prev = avg_cost
    if (epoch+1) % 20==0:
      print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.5f}'.format(avg_cost))
      
      

      
print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.5f}'.format(avg_cost))
      
print(hyp, realy)

Epoch: 0020 cost = 356.30275
Epoch: 0039 cost = 356.00859
[[88.61607 ]
 [88.81342 ]
 [88.98108 ]
 [88.66231 ]
 [88.72365 ]
 [88.790665]
 [88.67296 ]
 [88.730804]
 [88.891365]
 [88.8388  ]
 [88.97419 ]
 [88.911064]
 [88.52132 ]
 [88.54961 ]
 [88.740906]
 [88.74955 ]
 [88.76405 ]
 [88.835075]
 [89.025826]
 [89.001366]
 [88.919464]
 [88.8416  ]
 [88.813644]
 [88.9344  ]
 [88.68086 ]
 [88.77253 ]
 [88.8962  ]
 [88.86653 ]
 [88.73546 ]
 [88.89217 ]
 [88.615074]
 [88.743805]
 [88.71643 ]
 [88.85804 ]
 [88.926575]
 [88.744255]
 [88.90029 ]
 [88.68166 ]
 [88.909935]
 [88.7542  ]
 [88.733315]
 [88.89157 ]
 [88.80962 ]
 [88.68657 ]
 [88.59306 ]
 [88.87121 ]
 [88.808624]
 [88.82445 ]
 [88.68743 ]
 [88.580826]
 [88.56057 ]
 [88.88976 ]
 [88.84859 ]
 [88.820244]
 [88.92808 ]
 [88.735886]
 [88.566284]
 [88.822876]
 [88.92711 ]
 [88.76406 ]
 [88.734985]
 [88.81884 ]
 [88.763664]
 [88.81194 ]
 [88.72519 ]
 [88.91198 ]
 [88.74754 ]
 [88.86491 ]
 [88.68209 ]
 [88.62709 ]
 [88.92467 ]
 [88.829575]
 [88.8

In [129]:
# Test set 계산

y_test = y_test.reshape(-1,1)
mse = tf.reduce_mean(tf.square(hypothesis - Y))**0.5
print('Cost of test set:', sess.run(mse, feed_dict={
      X: X_test, Y: y_test, keep_prob:1, phase:False}))

Cost of test set: 22.429485
